<a href="https://colab.research.google.com/github/xhxuciedu/CS206/blob/master/krylov_subspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Krylov subspace methods**

Krylov subspace methods are a family of algorithms for solving $Ax=b$ that search for an approximate solution from a Krylov subspace. They are iterative, as opposed to direct, algorithms and usually require a fast matrix-vector product for $A$  (and possibly  $A^T$ ). The most famous Krylov subspace methods are Arnoldi, Lanczos, conjugate gradient, BiCGSTAB, and GMRES. We do not have time to cover them all. Here, we will focus on the GMRES method.



## **Initial thoughts on Krylov subspaces**

Let  $A$  be an invertible matrix and suppose we want to solve  $Ax=b$ , but we only know  $A$  via matrix-vector products, i.e., a function  $v\to Av$ . Let's consider the sequence of vectors  $b,Ab,…,A^{n−1}b,A^nb,…,$ 
 (not much else one can consider!). If we consider the first  $n+1$ , i.e.,  
 $$b,Ab,…,A^{n−1}b,A^nb$$ , then we are guaranteed for these vectors to be linear dependent ( $n+1$  vectors in  $n$-dimensional space must be linear dependent). Therefore, these exist coefficients  $α_0,…,α_n$  such that
$$α_0b+α_1Ab+⋯+α_nA^nb=0.$$
 
Let  $k$  be the smallest integer so that  $α_k≠0$ . Then, since  $A^{−1}$  exists:
$$A^{−1}b=−\frac{1}{α_k}(α_{k+1}b+⋯+α_nA^{n−k−1}b).$$
(Weak Cayley-Hamilton theorem). This shows that  $x=A^{−1}b$  can be computed by only matrix-vector products. This gives us an idea to search for "good" solutions from Krylov subspaces.

## **The Krylov subspace**
Let  $A$  be a matrix and  $c$  a vector. The  $r$th Krylov subspace, denoted by  $\mathcal{K}_r(A,c)$  is the vector spaced spanned by the vectors,
$$c,Ac,…,A^{r−1}c.$$
 
Usually, but not necessarily, the vector  $c$  is the right-hand side in a linear system  $Ax=b$  so that  $c=b$.



## **Generalized minimum residual method (GMRES)**
The GMRES is a Krylov subspace method that computes at the  $r$th step the best least squares solution  $x_r$  from the Krylov subspace  $\mathcal{K}_r(A,b)$. That is, the GMRES method successively solves the following least squares problems:

$$
\begin{array}[cc]
\textrm{Step 1} & \min_{x_1\in\mathcal{K}_1(A,b)}‖b−Ax_1‖_2,\\
\textrm{Step 2} & \min_{x_2\in\mathcal{K}_2(A,b)}‖b−Ax_2‖_2,\\
\vdots & \vdots\\
\textrm{Step r} & \min_{x_r\in\mathcal{K}_r(A,b)}‖b−Ax_r‖_2,\\
\end{array}
 $$

<br>

These problems may seem to be awkward to solve at first because we do not know to find the least squares solution over Krylov subspaces. However, if the columns of  Qr  form an orthonormal basis for  $\mathcal{K}_r(A,b)$, then we can equivalently write the GMRES method as

$$
\begin{array}[ccc]
\textrm{Step 1} & \min_{c\in\mathbb{R}}‖b−AQ_1c‖_2, & x_1=Q_1c,\\
\vdots & \vdots & \vdots\\
\textrm{Step 2} & \min_{c\in\mathbb{R^2}}‖b−AQ_2c‖_2, & x_2=Q_2c,\\
\textrm{Step r} & \min_{c\in\mathbb{R^r}}‖b−AQ_rc‖_2, & x_r=Q_rc,
\end{array}
 $$



## **Solving least squares**

Therefore, for the  $r$th GMRES step we need to find the least squares solution to  $AQ_rc=b$ . We do this by using the  QR  factorization of  $AQ_r$ , i.e.,  $AQ_r=\tilde Q_r\tilde R_r$ . Thus, we can rewrite GMRES again as the following steps:

$$
\begin{array}
\textrm{Step 1} & \mbox{Solve }\tilde R_1c=\tilde Q_1^Tb, & \tilde Q_1\tilde R_1=AQ_1,  & x_1=Q_1c,\\
\textrm{Step 2} & \mbox{Solve }\tilde R_2c=\tilde Q_2^Tb, & \tilde Q_2\tilde R_2=AQ_2, & x_2=Q_2c,\\
 \vdots & \vdots & \vdots &\vdots\\
\textrm{Step r} & \mbox{Solve }\tilde R_rc=\tilde Q_r^Tb, & \tilde Q_r\tilde R_r=AQ_r, & x_r=Q_rc
\end{array}
$$


<br>

The GMRES method solves the above steps, but does so recursively so that the  $r$th step uses the fact that the  $(r−1)$th step has already been computed.

There are three questions remaining:

* How to recursively construct an orthonormal basis for the Krylov subspaces?
* How to recursively compute the  QR  factorization of  $AQ_r$ ?
* How many steps do we need before  $‖x_r−A^{-1}b‖_2$  is small?

## How to recursively construct an orthonormal basis for the Krylov subspaces?

We want to compute the following  QR  factorization, which immediately constructs an orthonormal basis for  $\mathcal{K}_r(A,b)$ :
$$
Q_rR_r=[b | Ab|\cdots|A^{r−1}b].
$$

But, we cannot do this naively because it is numerically unstable due to the high matrix powers involved. Instead, a better idea is to construct the Krylov subspace iteratively as follows:
$$
\begin{align}
&Q_1=\frac{b}{\|b\|_2},\\
&Q_2R_2=[Q_1|AQ_1[:,-1]]\\
&Q_3R_3=[Q_2|AQ_2[:,-1]]\\
&\cdots\\
&Q_rR_r=[Q_{r−1}|AQ_{r−1}[:,-1]]
\end{align}
 $$

## **Arnoldi iterations**

Recall that an $m\times m$ matrix $A$ can be reduced to Hessenberg form by an orthogonal similarity transformation
$$
A = QHQ^T\quad\mbox{or}\quad AQ=QH
$$
Now consider the first $n$ columns of $AQ=QH$. Let $Q_n$ be the $m\times n$ matrix whose columns are the first $n$ columns of $Q$:
$$
Q_n = \begin{bmatrix}
| & | & & | \\
q_1& q_2 &\cdots & q_n\\
| & | & & | 
\end{bmatrix}
$$
Then we have 
$$
AQ_n = Q_{n+1}\tilde H_n
$$
where $\tilde H_n$ is the $(n+1)\times n$ upper-left section of $H$, whish is also a Hessenberg matrix
$$
\tilde H_n = 
\begin{bmatrix}
h_{11} & \cdots & & h_{1n} \\
h_{21}&h_{22}  &\cdots & \\
& \ddots & \ddots & \vdots\\
& & h_{n,n-1} & h_{nn}\\
& & & h_{n+1,n}
\end{bmatrix}
$$

The $n$th column of thie equation can be written down as follows:
$$
Aq_n = h_{1n}q_1 + \cdots + h_{nn}q_n + h_{n+1,n}q_{n+1}
$$

$q_{n_1}$ satisfies the recurrence relation involving itself and the previous Krylov vectors.  The Arnoldi iteration is simply the modified Gram-Schmidt iteration that implements the above equation. 